<a href="https://colab.research.google.com/github/2025ab05031/Sem1_ML_Assignment2/blob/main/2025ab05031_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>